In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install git+https://github.com/facebookresearch/segment-anything.git

  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-zaitiqqt
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-zaitiqqt
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Preparing metadata (setup.py) ... done
  Created wheel for segment-anything: filename=segment_anything-1.0-py3-none-any.whl size=36588 sha256=7ddeb4ed19cdab8328697206e8e3c3c21635eafbc560f44ec9c2403135e4ebac
  Stored in directory: /tmp/pip-ephem-wheel-cache-smhgnhhb/wheels/10/cf/59/9ccb2f0a1bcc81d4fbd0e501680b5d088d690c6cfbc02dc99d
Successfully built segment-anything


In [ ]:
!pip install opencv-python pycocotools matplotlib onnxruntime onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.7 MB/s eta 0:00:00


In [ ]:
using_colab = True

In [ ]:
if using_colab:
    import torch
    import torchvision
    print("PyTorch version:", torch.__version__)
    print("Torchvision version:", torchvision.__version__)
    print("CUDA is available:", torch.cuda.is_available())
    import sys
    !{sys.executable} -m pip install opencv-python matplotlib
    !{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'

    !mkdir images
    !wget -P images https://raw.githubusercontent.com/facebookresearch/segment-anything/main/notebooks/images/dog.jpg

    !wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

PyTorch version: 2.0.1+cu118
Torchvision version: 0.15.2+cu118
CUDA is available: False
  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-cj21sc6u
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-cj21sc6u
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Preparing metadata (setup.py) ... done
--2023-08-05 17:53:03--  https://raw.githubusercontent.com/facebookresearch/segment-anything/main/notebooks/images/dog.jpg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 99846 (98K) [image/jpeg]
Saving to: ‘images/dog.jpg’

dog.jpg             100%[===================>]  97.51K  -

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2

In [ ]:
import sys
sys.path.append("..")
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

sam_checkpoint = "sam_vit_h_4b8939.pth"
model_type = "vit_h"

device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

mask_generator = SamAutomaticMaskGenerator(sam)

RuntimeError: ignored

In [ ]:
def create_mask(image_path):
  image = cv2.imread(image_path)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  masks = mask_generator.generate(image)
  mask_img = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)
  for mask in masks:
    if mask['area'] < 50000:
      m = mask['segmentation']
      mask_img[m] = 255  # 将标注区域的像素值设为255，表示白色
  return mask_img

In [ ]:
input_folder = "./drive/MyDrive/CUHK/BreaKHis_allrandom/100X/train"
output_folder = "./drive/MyDrive/SAM/100X/train"

In [ ]:
def process_folder(src_folder, dst_folder):
    # 遍历源文件夹中的所有文件和子目录

    for filename in os.listdir(src_folder):
        src_file_path = os.path.join(src_folder, filename)
        # 判断是否是目录
        if os.path.isdir(src_file_path):
            # 如果是目录，创建对应的目标子目录
            new_dst_folder = os.path.join(dst_folder, filename)
            os.makedirs(new_dst_folder, exist_ok=True)
            # 递归调用process_folder处理子目录
            process_folder(src_file_path, new_dst_folder)
        else:
            # 如果是文件，判断是否是图像文件（可以根据实际情况扩展支持的图像格式）
            if filename.lower().endswith('.png'):
                # 读取图像数据
                # 为图像建立掩码
                mask = create_mask(src_file_path)

                # 将掩码保存为图像文件
                mask_filename = os.path.splitext(filename)[0] + "_mask.png"  # 生成掩码文件名
                dst_file_path = os.path.join(dst_folder, mask_filename)
                cv2.imwrite(dst_file_path, mask)

In [ ]:
import os

In [ ]:
process_folder(input_folder, output_folder)

In [ ]:
image_path = './drive/MyDrive/SAM/12.png'
cv2.imwrite("mask_output.png", create_mask(image_path))

True